In [1]:
import cv2
import motors
import time
import numpy as np
from ultralytics import YOLO
import ipywidgets.widgets as widgets
from IPython.display import display, clear_output

#Creating robot
robot = motors.MotorsYukon(mecanum=False)

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#Convert a NumPy array to JPEG-encoded data for display
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

# Load the YOLO model
model = YOLO("yolo11l_half.engine")
# model = YOLO("yolo11n.engine")

# Open the video file
video_path = "color_video.avi"
cap = cv2.VideoCapture(video_path)

# Set the video codec and save the processed video.
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 'mp4v' codec, suitable for MP4 files
width, height = 672,376 #VGA resolution
fps = 30
color_file = cv2.VideoWriter('color_video_processed.avi', fourcc, fps, (width, height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        t1 = cv2.getTickCount()
        # Run YOLO inference on the frame
        results = model(frame,verbose=False)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        color_file.write(annotated_frame)

        framed = frame.copy()
        #set a confidence threshold to filter out unconfident boxes
        #https://docs.ultralytics.com/modes/predict/#boxes
        conf_threshold = 0.5
        for result in results:
            #get the human subject
            for i in range (len(result.boxes.cls)):
                if(result.boxes.cls[i] == 0):  #human subject
                    # print(result.boxes.xywh[i])
                    if (result.boxes.conf[i] > conf_threshold): #you
                        # print()
                        bbox = result.boxes.xyxy[i]
                        # print(bbox)
                        cv2.rectangle(framed, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
                        # cv2.imwrite('human.jpg',color_img)
                
        scale = 0.3
        resized_image = cv2.resize(annotated_frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        resized_image2 = cv2.resize(framed, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_image)
        display_depth.value = bgr8_to_jpeg(resized_image2)
        
        # total_time = (cv2.getTickCount() - t1) / cv2.getTickFrequency()
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
color_file.release()

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [2]:
# You will need to load the YOLO model if you skip the first code block.
# from ultralytics import YOLO
# model = YOLO("yolo11l_half.engine")

#Start the camera system
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())   
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data

[2025-05-07 11:35:10 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:35:10 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:35:10 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:35:11 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-05-07 11:35:12 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-05-07 11:35:12 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-05-07 11:35:12 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-05-07 11:35:12 UTC][ZED][INFO] [Init]  Serial Number: S/N 39784002
Loading yolo11l_half.engine for TensorRT inference...


In [3]:
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display

display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')


# Variable to store the tracked person
tracked_person = None
last_seen_time = 0
selected_person_bbox = None #new
TRACK_TIMEOUT = 1.0  

person_dropdown = widgets.Dropdown(
    options=[('None', None)],  # Start with None option
    description='Select Person:',
    disabled=False
)

tracking_status = widgets.Label(value="Status: No person selected")
display(widgets.VBox([person_dropdown, tracking_status]))
display(sidebyside)

def func(change):
    global tracked_person
    
    frame = change['new']
    results = model(frame, verbose=False)
    
    # Get current persons in frame
    persons = []
    dropdown_options = [('None', None)]  # Always include None option
    
    for i in range(len(results[0].boxes.cls)):
        if results[0].boxes.cls[i] == 0 and results[0].boxes.conf[i] > 0.5:
            bbox = results[0].boxes.xyxy[i].cpu().numpy()
            persons.append(bbox)
            dropdown_options.append((f"Person {len(dropdown_options)}", bbox))
    
    # Update dropdown options if they've changed
    if dropdown_options != person_dropdown.options:
        with person_dropdown.hold_trait_notifications():
            person_dropdown.options = dropdown_options
    
    # Only track if a person is selected
    if person_dropdown.value is not None:
        # Find the selected person in current frame
        selected_person = None
        for bbox in persons:
            if np.array_equal(bbox, person_dropdown.value):
                selected_person = bbox
                break
        
        if selected_person is not None:
            tracked_person = selected_person
            distance = getPersonDepth(tracked_person, camera.depth_image)
            turn, move = generate_robot_commands(tracked_person, frame.shape[1], distance)
            execute_movement(turn, move)
            
            tracking_status.value = f"Status: Tracking selected person | Dist: {distance:.0f}mm"
            
            # Draw all persons
            for i, bbox in enumerate(persons):
                color = (0, 255, 0) if np.array_equal(bbox, tracked_person) else (255, 0, 0)
                cv2.rectangle(frame, 
                             (int(bbox[0]), int(bbox[1])),
                             (int(bbox[2]), int(bbox[3])),
                             color, 2)
                if not np.array_equal(bbox, tracked_person):
                    cv2.putText(frame, f"Person {i+1}", 
                              (int(bbox[0]), int(bbox[1]-10)),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            
            status = f"Tracking: {move} {turn}"
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            robot.stop()
            tracking_status.value = "Status: Selected person not found in frame"
    else:
        robot.stop()
        tracking_status.value = "Status: No person selected"
        
        # Draw all persons (but don't track)
        for i, bbox in enumerate(persons):
            cv2.rectangle(frame, 
                         (int(bbox[0]), int(bbox[1])),
                         (int(bbox[2]), int(bbox[3])),
                         (255, 0, 0), 2)
            cv2.putText(frame, f"Person {i+1}", 
                      (int(bbox[0]), int(bbox[1]-10)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    # Display
    resized_image = cv2.resize(frame, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_image)
    
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), cv2.COLORMAP_JET
    )
    resized_depth = cv2.resize(depth_colormap, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth)



camera.observe(func, names=['color_value'])
# camera.stop()

In [4]:
def execute_movement(turn, move):
    # Movement
    if move == 'forward':
        robot.forward(0.7) 
    elif move == 'backward':
        robot.backward(0.7)  
    else:
        robot.stop()
    
    if turn == 'left':
        robot.left(0.4)
    elif turn == 'right':
        robot.right(0.4)

In [5]:
def personSelection(results, image_center, conf_threshold = 0.5):
    minimum_distance = float('inf')
    selected_bbox = None
    
    for i in range(len(results[0].boxes.cls)):
        if results[0].boxes.cls[i] == 0 and results[0].boxes.conf[i] > conf_threshold:
            bbox = results[0].boxes.xyxy[i]
            center = (bbox[0]+bbox[2]) / 2
            distance = abs(center - image_center)
            if distance < minimum_distance:
                minimum_distance = distance
                selected_bbox = bbox
    return selected_bbox

In [6]:
def getPersonDepth(bbox, depth_image):
    centerX = int((bbox[0] + bbox[2]) / 2)
    centerY = int((bbox[1] + bbox[3]) / 2)
    depth = depth_image[centerY, centerX]
    return depth

In [7]:
def generate_robot_commands(bbox, frame_width, person_distance, desired_distance=1000):
    center_x = (bbox[0] + bbox[2]) / 2
    error_x = center_x - (frame_width / 2)
    
    norm_distance_error = abs(person_distance - desired_distance) / desired_distance
    norm_position_error = abs(error_x) / (frame_width / 2)
    
    distance_dominant_threshold = 0.3  
    position_dominant_threshold = 0.2  
    

    if norm_distance_error > distance_dominant_threshold:
        if person_distance < desired_distance - 200:  
            move = 'backward'
            if person_distance > 500:  
                if norm_position_error > 0.3:
                    turn = 'right' if error_x > 0 else 'left'
                else:
                    turn = 'none'
            else:
                turn = 'none'  
        else:
            if norm_position_error > position_dominant_threshold:
                turn = 'right' if error_x > 0 else 'left'
            else:
                turn = 'none'
            move = 'forward' if person_distance > desired_distance + 200 else 'stop'
    else:
        if norm_position_error > position_dominant_threshold:
            turn = 'right' if error_x > 0 else 'left'
        else:
            turn = 'none'
        move = 'forward' if person_distance > desired_distance + 200 else \
              'backward' if person_distance < desired_distance - 200 else 'stop'
    
    # When moving backward, reduce turning sensitivity
    if move == 'backward':
        if norm_position_error > 0.4:  # Higher threshold when backing up
            turn = 'right' if error_x > 0 else 'left'
        else:
            turn = 'none'
    
    return turn, move

[05/07/2025-12:35:13] [TRT] [I] Loaded engine size: 52 MiB
[05/07/2025-12:35:13] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[05/07/2025-12:35:13] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +36, now: CPU 1, GPU 84 (MiB)
